In [91]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3


ERROR: Too many arguments


^C


In [4]:
import os
import requests
from dotenv import load_dotenv
import json
import pandas as pd

load_dotenv()

True

# API Calls

In [12]:
API_KEY = os.environ.get("API_KEY")

headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

def get_movie():
    url = "https://api.themoviedb.org/3/movie/12?language=en-US"
    response = requests.get(url, headers=headers)

    return response.text

def get_genres():
    url = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    response = requests.get(url, headers=headers)

    return response.text

def get_series_genres():
    url = "https://api.themoviedb.org/3/genre/tv/list?language=en"
    response = requests.get(url, headers=headers)

    return response.text

def popular_movies(page_num):
    url = f"https://api.themoviedb.org/3/movie/popular?language=en&page={page_num}"
    response = requests.get(url, headers=headers)

    return response.text

def popular_series(page_num):
    url = f"https://api.themoviedb.org/3/tv/popular?language=en-US&page={page_num}"
    response = requests.get(url, headers=headers)

    return response.text


def get_movie_by_genres(page_num, genres):
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page_num}&sort_by=popularity.desc&with_genres={genres}"
    response = requests.get(url, headers=headers)

    return response.text

def get_tv_shows_by_genres(page_num, genres):
    url = f"https://api.themoviedb.org/3/discover/tv?include_adult=false&include_null_first_air_dates=false&language=en-US&page={page_num}&sort_by=popularity.desc&with_genres={genres}"
    response = requests.get(url, headers=headers)

    return response.text

# Get Movies

In [6]:
json_string = get_genres()
data = json.loads(json_string)

genres_dict = {genre["id"]: genre["name"] for genre in data["genres"]}

genres_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [7]:
movies_csv_file = pd.DataFrame(columns=['Name','overview', 'genres', 'language', "popularity"]) 
movies_csv_file

,Name,overview,genres,language,popularity


In [13]:
genres_dict.keys()
genre_ids_list = list(genres_dict)

pages = 501
for idx, genre_id in enumerate(genre_ids_list):
    for num in range(1, pages): 
        try:     
            movies = get_movie_by_genres(page_num=num, genres=genre_id)
            movies_json = json.loads(movies)
            
            for movie in movies_json["results"]:
                genres = ", ".join([genres_dict.get(genre_id, "Unknown") for genre_id in movie["genre_ids"]])

                movie_data = pd.DataFrame([{
                    'Name': movie["original_title"],
                    'overview': movie["overview"],
                    'genres': genres,
                    'language': movie["original_language"],
                    'popularity': movie["popularity"]
                }])
                
                if not ((movies_csv_file['Name'] == movie_data.iloc[0]['Name']) & (movies_csv_file['overview'] == movie_data.iloc[0]['overview'])).any():
                    # If the movie doesn't exist, append the data
                    movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)

            print(f"----- Page {num} are added. -----")
        except Exception as e:
            print("----- Data is unavailable -----", e)

movies_csv_file.head()
movies_csv_file.to_csv('movies_genres.csv', index=False)
print("movies_genres.csv file has been created")

C:\Users\Asus\AppData\Local\Temp/ipykernel_23824/1072221689.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)


----- Page 1 are added. -----
----- Page 2 are added. -----
----- Page 3 are added. -----
----- Page 4 are added. -----
----- Page 5 are added. -----
----- Page 6 are added. -----
----- Page 7 are added. -----
----- Page 8 are added. -----
----- Page 9 are added. -----
----- Page 10 are added. -----
----- Page 11 are added. -----
----- Page 12 are added. -----
----- Page 13 are added. -----
----- Page 14 are added. -----
----- Page 15 are added. -----
----- Page 16 are added. -----
----- Page 17 are added. -----
----- Page 18 are added. -----
----- Page 19 are added. -----
----- Page 20 are added. -----
----- Page 21 are added. -----
----- Page 22 are added. -----
----- Page 23 are added. -----
----- Page 24 are added. -----
----- Page 25 are added. -----
----- Page 26 are added. -----
----- Page 27 are added. -----
----- Page 28 are added. -----
----- Page 29 are added. -----
----- Page 30 are added. -----
----- Page 31 are added. -----
----- Page 32 are added. -----
----- Page 33 are

# Get Series

In [13]:
series_json_string = get_series_genres()
series_data = json.loads(series_json_string)

series_genres_dict = {genre["id"]: genre["name"] for genre in series_data["genres"]}
series_genres_dict

{10759: 'Action & Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 10762: 'Kids',
 9648: 'Mystery',
 10763: 'News',
 10764: 'Reality',
 10765: 'Sci-Fi & Fantasy',
 10766: 'Soap',
 10767: 'Talk',
 10768: 'War & Politics',
 37: 'Western'}

In [18]:
tv_shows_csv_file = pd.DataFrame(columns=['Name','overview', 'genres', 'language', "popularity"]) 
tv_shows_csv_file

,Name,overview,genres,language,popularity


In [22]:
series_genres_dict.keys()
series_genre_ids_list = list(series_genres_dict)

pages = 501
for idx, genre_id in enumerate(series_genre_ids_list):
    for num in range(1, pages): 
        try:     
            tv_shows = get_tv_shows_by_genres(page_num=num, genres=genre_id)
            tv_shows_json = json.loads(tv_shows)
            
            for tv_show in tv_shows_json["results"]:
                genres = ", ".join([genres_dict.get(genre_id, "Unknown") for genre_id in tv_show["genre_ids"]])

                tv_show_data = pd.DataFrame([{
                    'Name': tv_show["original_name"],
                    'overview': tv_show["overview"],
                    'genres': genres,
                    'language': tv_show["original_language"],
                    'popularity': tv_show["popularity"]
                }])
                
                if not ((tv_shows_csv_file['Name'] == tv_show_data.iloc[0]['Name']) & (tv_shows_csv_file['overview'] == tv_show_data.iloc[0]['overview'])).any():
                    tv_shows_csv_file = pd.concat([tv_shows_csv_file, tv_show_data], ignore_index=True)

            if num % 250 == 0:
                print(f"----- Page {num} of {series_genres_dict[genre_id]} are added. -----")

        except Exception as e:
            print("----- Data is unavailable -----", e)
        
        print(f"----- {series_genres_dict[genre_id]} genre are completely added. -----")

tv_shows_csv_file.head()
tv_shows_csv_file.to_csv('tv_shows_genres.csv', index=False)
print("tv_shows_genres.csv file has been created")

----- Page 250 of Western are added. -----
----- Page 500 of Western are added. -----
----- Western genre are completely added. -----
tv_shows_genres.csv file has been created


In [24]:
tv_shows_csv_file.head()

,Name,overview,genres,language,popularity
0,淮水竹亭,It tells the story of Dongfang Huaizhu and Wan...,"Drama, Unknown, Unknown",zh,709.2099
1,Ang Probinsyano,Ador is a well known and respected CIDG Police...,"Drama, Unknown",tl,227.5022
2,ドラえもん,Robotic cat Doraemon is sent back in time from...,"Unknown, Animation, Comedy, Unknown, Unknown, ...",ja,211.7783
3,Game of Thrones,Seven noble families fight for control of the ...,"Unknown, Drama, Unknown",en,177.6111
4,약한영웅,"With the aid of unexpected friends, a gifted b...","Unknown, Drama",ko,174.6151


# Preprocessing

In [33]:
movies_csv_file['popularity'] = pd.to_numeric(movies_csv_file['popularity'], errors='coerce')
movies_csv_file['Name'] = movies_csv_file['Name'].str.strip().str.lower()

tv_shows_csv_file['popularity'] = pd.to_numeric(tv_shows_csv_file['popularity'], errors='coerce')
tv_shows_csv_file['Name'] = tv_shows_csv_file['Name'].str.strip().str.lower()

In [39]:
movies_csv_file = pd.read_csv('movies_genres.csv')
movies_csv_file.fillna('', inplace=True)

movies_csv_file['combined_text'] = (
    'Title: ' + movies_csv_file['Name'].astype(str) + '. ' +
    'Overview: ' + movies_csv_file['overview'].astype(str) + '. ' +
    'Genres: ' + movies_csv_file['genres'].astype(str) + '. ' +
    'Language: ' + movies_csv_file['language'].astype(str) + '. ' +
    'Popularity: ' + movies_csv_file['popularity'].astype(str)
)

movies_csv_file.head()

,Name,overview,genres,language,popularity,combined_text
0,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,575.1039,Title: Havoc. Overview: When a drug heist swer...
1,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,548.9128,Title: A Working Man. Overview: Levon Cade lef...
2,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,276.1329,Title: 新幹線大爆破. Overview: When panic erupts on ...
3,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,275.3451,Title: In the Lost Lands. Overview: A queen se...
4,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,273.4631,Title: ज्वेल थीफ: द हीस्ट बिगिन्स. Overview: I...


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')  # مدل سبک و سریع
embeddings = model.encode(movies_csv_file['combined_text'].tolist(), show_progress_bar=True)

print(f"✅ {len(embeddings)} embedding ساخته شد.")

# ---------- مرحله 5: ذخیره در Faiss ----------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

faiss.write_index(index, "movie_index.faiss")

print("✅ embeddingها در فایل movie_index.faiss ذخیره شدند.")
